In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

# Criando banco

In [2]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_JUNK_DEMOGRAFICO" (
	"SK_DEMOGRAFICO" INTEGER NOT NULL UNIQUE,
	"DS_SEXO" VARCHAR,
	"DS_FAIXA_ETARIA" VARCHAR,
  "DT_CARGA" TEXT,
	PRIMARY KEY("SK_DEMOGRAFICO")
)''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_LOCAL_OBITO" (
	"SK_LOCAL_OBITO" INTEGER NOT NULL UNIQUE,
	"CD_LOCAL_OBITO" INTEGER,
	"DS_LOCAL_OBITO" VARCHAR,
  "DT_CARGA" TEXT,
	PRIMARY KEY("SK_LOCAL_OBITO")
)''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_OCUPACAO" (
	"SK_OCUPACAO" INTEGER NOT NULL UNIQUE,
	"CD_CBO" INTEGER,
	"DS_OCUPACAO" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_OCUPACAO")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_DOENCA" (
	"SK_DOENCA" INTEGER NOT NULL UNIQUE,
	"CD_CID" VARCHAR,
	"DS_DOENCA" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_DOENCA")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_CIRCUNSTANCIA" (
	"SK_CIRCUNSTANCIA" INTEGER NOT NULL UNIQUE,
	"CD_CIRCUNSTANCIA" INTEGER,
	"DS_CIRCUNSTANCIA" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_CIRCUNSTANCIA")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_ESTADO" (
    "SK_ESTADO" INTEGER NOT NULL UNIQUE,
    "CD_ESTADO" INTEGER,
    "CD_UF" INTEGER,
    "DS_ESTADO" VARCHAR,
    "DS_REGIAO" VARCHAR,
		"DT_CARGA" TEXT,
    PRIMARY KEY("SK_ESTADO")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_TEMPO_MES" (
	"SK_TEMPO_MES" INTEGER NOT NULL UNIQUE,
	"DS_ANO_MES" VARCHAR,
	"NR_ANO" INTEGER,
	"NR_MES" INTEGER,
	"DS_MES" VARCHAR,
	"NR_TRIMESTRE" INTEGER,
	"NR_SEMESTRE" INTEGER,
	"NR_QTD_DIAS_MES" INTEGER,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_TEMPO_MES")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_JUNK_DETALHES" (
	"SK_DETALHES" INTEGER NOT NULL UNIQUE,
	"ST_ASSISTENCIA_MEDICA" VARCHAR,
	"ST_INVESTIGADO" VARCHAR,
	"ST_EPIDEMIA" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_DETALHES")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "FATO_OBITO_COVID" (
	"SK_TEMPO_MES" INTEGER,
	"SK_ESTADO" INTEGER,
	"SK_OCUPACAO" INTEGER,
	"SK_DEMOGRAFICO" INTEGER,
	"SK_DETALHES" INTEGER,
	"SK_LOCAL_OBITO" INTEGER,
	"SK_TERMINAL_1" INTEGER,
	"QT_QUANTIDADE" INTEGER NOT NULL,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_TEMPO_MES", "SK_ESTADO", "SK_OCUPACAO", "SK_DEMOGRAFICO", "SK_DETALHES", "SK_LOCAL_OBITO", "SK_TERMINAL_1")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "FATO_OBITO_IMPACTO" (
	"SK_TEMPO_MES" INTEGER NOT NULL,
	"SK_DETALHES" INTEGER,
	"SK_CIRCUNSTANCIA" INTEGER,
	"SK_LOCAL_OBITO" INTEGER,
	"SK_TERMINAL" INTEGER,
	"QT_QUANTIDADE" INTEGER,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_TEMPO_MES", "SK_DETALHES", "SK_CIRCUNSTANCIA", "SK_LOCAL_OBITO", "SK_TERMINAL")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "FATO_OBITO_PERFIL" (
	"SK_TEMPO_MES" INTEGER NOT NULL,
	"SK_TERMINAL_1" INTEGER,
	"SK_TERMINAL_2" INTEGER,
	"SK_BASICA_1" INTEGER,
	"SK_BASICA_2" INTEGER,
	"SK_DEMOGRAFICO" INTEGER,
	"SK_ESTADO" INTEGER,
	"ST_COVID" BOOLEAN,
	"QT_QUANTIDADE" INTEGER,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_TEMPO_MES", "SK_TERMINAL_1", "SK_TERMINAL_2", "SK_BASICA_1", "SK_BASICA_2", "SK_DEMOGRAFICO", "SK_ESTADO")
)''')

conn.commit()
conn.close()

In [52]:
# Conexão com o banco de dados
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()



# 2. Obter a lista de tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nNúmero de linhas em cada tabela:")

# 3. Contar as linhas em cada tabela
for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    row_count = cursor.fetchone()[0]
    print(f"Tabela '{table_name}': {row_count} linhas")

# Fechar a conexão
conn.close()


Número de linhas em cada tabela:
Tabela 'DIME_JUNK_DEMOGRAFICO': 27 linhas
Tabela 'DIME_LOCAL_OBITO': 8 linhas
Tabela 'DIME_OCUPACAO': 2796 linhas
Tabela 'DIME_DOENCA': 14234 linhas
Tabela 'DIME_CIRCUNSTANCIA': 6 linhas
Tabela 'DIME_ESTADO': 28 linhas
Tabela 'DIME_TEMPO_MES': 84 linhas
Tabela 'DIME_JUNK_DETALHES': 7 linhas
Tabela 'FATO_OBITO_COVID': 345522 linhas
Tabela 'FATO_OBITO_IMPACTO': 522438 linhas
Tabela 'FATO_OBITO_PERFIL': 5623812 linhas


# Inserindo Carga Manual na table DIME_JUNK_DEMOGRAFICO

In [4]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Lista de combinações (DS_SEXO, DS_FAIXA_ETARIA)
dados = [
    ("Masculino", "Recém-Nascido"),
    ("Feminino", "Recém-Nascido"),
    ("Masculino", "Bebê"),
    ("Feminino", "Bebê"),
    ("Masculino", "Criança"),
    ("Feminino", "Criança"),
    ("Masculino", "Adolescente"),
    ("Feminino", "Adolescente"),
    ("Masculino", "Jovem Adulto"),
    ("Feminino", "Jovem Adulto"),
    ("Masculino", "Adulto"),
    ("Feminino", "Adulto"),
    ("Masculino", "Idoso"),
    ("Feminino", "Idoso"),
    ("Masculino", "Mais de 100 anos"),
    ("Feminino", "Mais de 100 anos"),
    ("Ignorado", "Recém-Nascido"),
    ("Ignorado", "Bebê"),
    ("Ignorado", "Criança"),
    ("Ignorado", "Adolescente"),
    ("Ignorado", "Jovem Adulto"),
    ("Ignorado", "Adulto"),
    ("Ignorado", "Idoso"),
    ("Ignorado", "Mais de 100 anos"),
    ("Masculino", "Não Informado"),
    ("Feminino", "Não Informado"),
    ("Ignorado", "Não Informado"),
]

# Inserir sem duplicar (caso já exista)
for sexo, faixa in dados:
    cursor.execute("""
        SELECT 1 FROM DIME_JUNK_DEMOGRAFICO
        WHERE DS_SEXO = ? AND DS_FAIXA_ETARIA = ?
    """, (sexo, faixa))
    if not cursor.fetchone():
        cursor.execute("""
            INSERT INTO DIME_JUNK_DEMOGRAFICO (DS_SEXO, DS_FAIXA_ETARIA, DT_CARGA)
            VALUES (?, ?, ?)
        """, (sexo, faixa, data_carga))

# Commit e fechar
conn.commit()
conn.close()

print("✅ Dados inseridos/atualizados com sucesso!")


✅ Dados inseridos/atualizados com sucesso!


# Inserindo DIME_LOCAL_OBITO

In [5]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""

    SELECT SK_LOCAL_OBITO, CD_LOCAL_OBITO, DS_LOCAL_OBITO
    FROM DWCD_LOCAL_OBITO
    WHERE ST_SITUACAO = 'A'
""")
local_obito = cursor.fetchall()

df_local_obito = pd.DataFrame(local_obito, columns=['SK_LOCAL_OBITO', 'CD_LOCAL_OBITO', 'DS_LOCAL_OBITO'])

conn.commit()
conn.close()

In [6]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_LOCAL_OBITO
    """
)

for row in df_local_obito.itertuples(index=False):
  sk_local_obito, cd_local_obito, ds_local_obito = row
  print(sk_local_obito, cd_local_obito, ds_local_obito)
  cursor.execute(
      """
      INSERT INTO DIME_LOCAL_OBITO (SK_LOCAL_OBITO, CD_LOCAL_OBITO, DS_LOCAL_OBITO, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_local_obito, cd_local_obito, ds_local_obito, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 1 Hospital
2 2 outros estabelecimentos de saúde
3 3 Domicílio
4 4 Via pública
5 5 Outros
6 6 Aldeia indígena
7 9 Ignorado


# Inserindo na tabela DIME_OCUPACAO

In [7]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_OCUPACAO, CD_CBO, DS_OCUPACAO
    FROM DWCD_OCUPACAO
    WHERE ST_SITUACAO = 'A'
""")
ocupacao = cursor.fetchall()

df_ocupacao = pd.DataFrame(ocupacao, columns=['SK_OCUPACAO', 'CD_CBO', 'DS_OCUPACAO'])

conn.commit()
conn.close()

In [8]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_OCUPACAO
    """
)

for row in df_ocupacao.itertuples(index=False):
  sk_ocupacao, cd_cbo, ds_ocupacao = row
  print(sk_ocupacao, cd_cbo, ds_ocupacao)
  cursor.execute(
      """
      INSERT INTO DIME_OCUPACAO (SK_OCUPACAO, CD_CBO, DS_OCUPACAO, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_ocupacao, cd_cbo, ds_ocupacao, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 10105 Oficial general da aeronáutica
2 10110 Oficial general do exército
3 10115 Oficial general da marinha
4 10205 Oficial da aeronáutica
5 10210 Oficial do exército
6 10215 Oficial da marinha
7 10305 Praça da aeronáutica
8 10310 Praça do exército
9 10315 Praça da marinha
10 20105 Coronel da polícia militar
11 20110 Tenente-coronel da polícia militar
12 20115 Major da polícia militar
13 20205 Capitão da polícia militar
14 20305 Primeiro tenente de polícia militar
15 20310 Segundo tenente de polícia militar
16 21105 Subtenente da policia militar
17 21110 Sargento da policia militar
18 21205 Cabo da polícia militar
19 21210 Soldado da polícia militar
20 30105 Coronel bombeiro militar
21 30110 Major bombeiro militar
22 30115 Tenente-coronel bombeiro militar
23 30205 Capitão bombeiro militar
24 30305 Tenente do corpo de bombeiros militar
25 31105 Subtenente bombeiro militar
26 31110 Sargento bombeiro militar
27 31205 Cabo bombeiro militar
28 31210 Soldado bombeiro mi

# Inserindo na tabela DIME_DOENCA

In [9]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_DOENCA, CD_CID, DS_DOENCA
    FROM DWCD_DOENCA
    WHERE ST_SITUACAO = 'A'
""")
doenca = cursor.fetchall()

df_doenca = pd.DataFrame(doenca, columns=['SK_DOENCA', 'CD_CID', 'DS_DOENCA'])

conn.commit()
conn.close()

In [10]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_DOENCA
    """
)

for row in df_doenca.itertuples(index=False):
  sk_doenca, cd_cid, ds_doenca = row
  print(sk_doenca, cd_cid, ds_doenca)
  cursor.execute(
      """
      INSERT INTO DIME_DOENCA (SK_DOENCA, CD_CID, DS_DOENCA, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_doenca, cd_cid, ds_doenca, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


A saída de streaming foi truncada nas últimas 5000 linhas.
9234 T236 Corrosão de Segundo Grau do Punho e da Mão
9235 T237 Corrosão de Terceiro Grau do Punho e da Mão
9236 T24 Queimadura e Corrosão do Quadril e Membro Inferior, Exceto Tornozelo e do pé
9237 T240 Queimadura do Quadril e Membro Inferior, Exceto Tornozelo e do Pé, Grau Não Especificado
9238 T241 Queimadura de Primeiro Grau do Quadril e do Membro Inferior, Exceto Tornozelo e do pé
9239 T242 Queimadura de Segundo Grau do Quadril e do Membro Inferior, Exceto Tornozelo e do pé
9240 T243 Queimadura de Terceiro Grau do Quadril e do Membro Inferior, Exceto Tornozelo e do pé
9241 T244 Corrosão do Quadril e do Membro Inferior, Exceto Tornozelo e do Pé, Grau Não Especificado
9242 T245 Corrosão de Primeiro Grau do Quadril e do Membro Inferior, Exceto Tornozelo e do pé
9243 T246 Corrosão de Segundo Grau do Quadril e do Membro Inferior, Exceto Tornozelo e do pé
9244 T247 Corrosão de Terceiro Grau do Quadril e do Membro Inferior, Exceto

# Inserindo na tabela DIME_CIRCUNSTANCIA

In [11]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_CIRCUSTANCIA, CD_CIRCUNSTANCIA, DS_CIRCUNSTANCIA
    FROM DWCD_CIRCUNSTANCIA
    WHERE ST_SITUACAO = 'A'
""")
circunstancia = cursor.fetchall()

df_circunstancia = pd.DataFrame(circunstancia, columns=['SK_CIRCUSTANCIA', 'CD_CIRCUNSTANCIA', 'DS_CIRCUNSTANCIA'])

conn.commit()
conn.close()

In [12]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_CIRCUNSTANCIA
    """
)

for row in df_circunstancia.itertuples(index=False):
  sk_circunstancia, cd_circunstancia, ds_circunstancia = row
  print(sk_circunstancia, cd_circunstancia, ds_circunstancia)
  cursor.execute(
      """
      INSERT INTO DIME_CIRCUNSTANCIA (SK_CIRCUNSTANCIA, CD_CIRCUNSTANCIA, DS_CIRCUNSTANCIA, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_circunstancia, cd_circunstancia, ds_circunstancia, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 1 Acidente
2 2 Suicídio
3 3 Homicídio
4 4 Outros
5 9 Ignorado


# Inserindo na tabela DIME_ESTADO

In [13]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_ESTADO, CD_ESTADO, CD_UF, DS_ESTADO, DS_REGIAO
    FROM DWCD_ESTADO
    WHERE ST_SITUACAO = 'A'
""")
estado = cursor.fetchall()

df_estado = pd.DataFrame(estado, columns=['SK_ESTADO', 'CD_ESTADO', 'CD_UF', 'DS_ESTADO', 'DS_REGIAO'])

conn.commit()
conn.close()

In [14]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_ESTADO
    """
)

for row in df_estado.itertuples(index=False):
  sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao = row
  print(sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao)
  cursor.execute(
      """
      INSERT INTO DIME_ESTADO (SK_ESTADO, CD_ESTADO, CD_UF, DS_ESTADO, DS_REGIAO, DT_CARGA)
      VALUES (?, ?, ?, ?,?,?)
      """,(sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado Não informado Não informado
1 11 RO Rondônia Norte
2 12 AC Acre Norte
3 13 AM Amazonas Norte
4 14 RR Roraima Norte
5 15 PA Pará Norte
6 16 AP Amapá Norte
7 17 TO Tocantins Norte
8 21 MA Maranhão Nordeste
9 22 PI Piauí Nordeste
10 23 CE Ceará Nordeste
11 24 RN Rio Grande do Norte Nordeste
12 25 PB Paraíba Nordeste
13 26 PE Pernambuco Nordeste
14 27 AL Alagoas Nordeste
15 28 SE Sergipe Nordeste
16 29 BA Bahia Nordeste
17 31 MG Minas Gerais Sudeste
18 32 ES Espírito Santo Sudeste
19 33 RJ Rio de Janeiro Sudeste
20 35 SP São Paulo Sudeste
21 41 PR Paraná Sul
22 42 SC Santa Catarina Sul
23 43 RS Rio Grande do Sul Sul
24 50 MS Mato Grosso do Sul Centro-Oeste
25 51 MT Mato Grosso Centro-Oeste
26 52 GO Goiás Centro-Oeste
27 53 DF Distrito Federal Centro-Oeste


# Extração de dados do DW

In [15]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT strftime('%Y-%m', DT_OBITO) AS TEMPO_MES, ST_ASSISTENCIA_MEDICA, ST_INVESTIGADO, ST_EPIDEMIA, SK_CIRCUSNTANCIA, SK_LOCAL_OBITO, SK_TERMINAL_1, COUNT(CD_OBITO) AS QTD_OBITOS
    FROM DWMV_OBITO
    GROUP BY TEMPO_MES, ST_ASSISTENCIA_MEDICA, ST_INVESTIGADO, ST_EPIDEMIA, SK_CIRCUSNTANCIA, SK_LOCAL_OBITO, SK_TERMINAL_1
    ORDER BY TEMPO_MES
""")
fato_impacto = cursor.fetchall()

df_fato_impacto = pd.DataFrame(fato_impacto, columns=['TEMPO_MES', 'ST_ASSISTENCIA_MEDICA', 'ST_INVESTIGADO', 'ST_EPIDEMIA', 'SK_CIRCUNSTANCIA','SK_LOCAL_OBITO', 'SK_TERMINAL_1', 'QTD_OBITOS'])

conn.commit()
conn.close()

In [16]:
df_fato_impacto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522438 entries, 0 to 522437
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   TEMPO_MES              522438 non-null  object
 1   ST_ASSISTENCIA_MEDICA  522438 non-null  object
 2   ST_INVESTIGADO         522438 non-null  object
 3   ST_EPIDEMIA            522438 non-null  object
 4   SK_CIRCUNSTANCIA       522438 non-null  int64 
 5   SK_LOCAL_OBITO         522438 non-null  int64 
 6   SK_TERMINAL_1          522438 non-null  int64 
 7   QTD_OBITOS             522438 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 31.9+ MB


In [17]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT
        strftime('%Y-%m', DT_OBITO) AS TEMPO_MES,
        SK_TERMINAL_1, SK_TERMINAL_2, SK_BASICA_1, SK_BASICA_2,
        S.DS_SEXO,
        CASE
            WHEN DWMV_OBITO.SK_TIPO_IDADE IN (1, 2, 3) THEN 'Recém-Nascido'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 4 THEN 'Bebê'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE < 5 THEN 'Bebê'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 5 AND 11 THEN 'Criança'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 12 AND 17 THEN 'Adolescente'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 18 AND 30 THEN 'Jovem Adulto'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 31 AND 59 THEN 'Adulto'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 60 AND 99 THEN 'Idoso'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 6 THEN 'Mais de 100 anos'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = -1 THEN 'Não Informado'
        END AS DS_FAIXA_ETARIA,
        M.SK_ESTADO,
        COUNT(CD_OBITO) AS QTD_OBITOS
    FROM DWMV_OBITO
    LEFT JOIN DWCD_MUNICIPIO M ON M.SK_MUNICIPIO = DWMV_OBITO.SK_MUNICIPIO_OBITO
    LEFT JOIN DWCD_SEXO S ON S.SK_SEXO = DWMV_OBITO.SK_SEXO
    GROUP BY TEMPO_MES, SK_TERMINAL_1, SK_TERMINAL_2, SK_BASICA_1, SK_BASICA_2,
             S.DS_SEXO, DS_FAIXA_ETARIA, M.SK_ESTADO
    ORDER BY TEMPO_MES
""")

fato_perfil = cursor.fetchall()

df_fato_perfil = pd.DataFrame(fato_perfil, columns=[
    'TEMPO_MES', 'SK_TERMINAL_1', 'SK_TERMINAL_2', 'SK_BASICA_1', 'SK_BASICA_2',
    'DS_SEXO', 'DS_FAIXA_ETARIA', 'SK_ESTADO', 'QTD_OBITOS'
])

conn.commit()
conn.close()


In [18]:
df_fato_perfil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5623812 entries, 0 to 5623811
Data columns (total 9 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   TEMPO_MES        object
 1   SK_TERMINAL_1    int64 
 2   SK_TERMINAL_2    int64 
 3   SK_BASICA_1      int64 
 4   SK_BASICA_2      int64 
 5   DS_SEXO          object
 6   DS_FAIXA_ETARIA  object
 7   SK_ESTADO        int64 
 8   QTD_OBITOS       int64 
dtypes: int64(6), object(3)
memory usage: 386.2+ MB


In [19]:
df_fato_perfil.sort_values(by=['QTD_OBITOS'], ascending=False)

,TEMPO_MES,SK_TERMINAL_1,SK_TERMINAL_2,SK_BASICA_1,SK_BASICA_2,DS_SEXO,DS_FAIXA_ETARIA,SK_ESTADO,QTD_OBITOS
2459536,2021-03,595,-1,595,595,Masculino,Idoso,20,1626
2477236,2021-03,4376,-1,595,595,Masculino,Idoso,20,1575
2477137,2021-03,4376,-1,595,595,Feminino,Idoso,20,1563
2459437,2021-03,595,-1,595,595,Feminino,Idoso,20,1552
2525041,2021-04,226,-1,595,595,Masculino,Idoso,20,1463
...,...,...,...,...,...,...,...,...,...
2077991,2020-09,4378,-1,1091,1091,Feminino,Idoso,27,1
2077990,2020-09,4378,-1,1091,1091,Feminino,Adulto,17,1
2077989,2020-09,4378,-1,1090,1090,Masculino,Idoso,21,1
2077987,2020-09,4378,-1,1090,1090,Masculino,Adulto,26,1


In [20]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_DOENCA
    FROM DIME_DOENCA
    WHERE CD_CID = 'B342'
""")

sk_coronavirus = cursor.fetchone()

conn.close()

if sk_coronavirus:
    print(f"O SK_DOENCA para 'B342' (Coronavirus) é: {sk_coronavirus[0]}")
else:
    print("Não foi encontrado SK_DOENCA para 'B342'.")

O SK_DOENCA para 'B342' (Coronavirus) é: 595


In [21]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT strftime('%Y-%m', DT_OBITO) AS TEMPO_MES, M.SK_ESTADO, SK_OCUPACAO, S.DS_SEXO, CASE
            WHEN DWMV_OBITO.SK_TIPO_IDADE IN (1, 2, 3) THEN 'Recém-Nascido'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 4 THEN 'Bebê'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE < 5 THEN 'Bebê'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 5 AND 11 THEN 'Criança'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 12 AND 17 THEN 'Adolescente'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 18 AND 30 THEN 'Jovem Adulto'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 31 AND 59 THEN 'Adulto'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 5 AND IDADE BETWEEN 60 AND 99 THEN 'Idoso'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = 6 THEN 'Mais de 100 anos'
            WHEN DWMV_OBITO.SK_TIPO_IDADE = -1 THEN 'Não Informado'
        END AS DS_FAIXA_ETARIA,
        ST_ASSISTENCIA_MEDICA, ST_INVESTIGADO, ST_EPIDEMIA, SK_LOCAL_OBITO, SK_TERMINAL_1, COUNT(CD_OBITO) AS QTD_OBITOS
    FROM DWMV_OBITO
    LEFT JOIN DWCD_MUNICIPIO M ON M.SK_MUNICIPIO = DWMV_OBITO.SK_MUNICIPIO_OBITO
    LEFT JOIN DWCD_SEXO S ON S.SK_SEXO = DWMV_OBITO.SK_SEXO
    WHERE SK_BASICA_1 = '595'
    GROUP BY TEMPO_MES, M.SK_ESTADO, SK_OCUPACAO, S.DS_SEXO, DS_FAIXA_ETARIA, ST_ASSISTENCIA_MEDICA, ST_INVESTIGADO, ST_EPIDEMIA, SK_LOCAL_OBITO, SK_TERMINAL_1
    ORDER BY TEMPO_MES
""")
fato_covid = cursor.fetchall()

df_fato_covid = pd.DataFrame(fato_covid, columns=['TEMPO_MES', 'M.SK_ESTADO', 'SK_OCUPACAO', 'DS_SEXO', 'DS_FAIXA_ETARIA', 'ST_ASSISTENCIA', 'ST_INVESTIGACAO', 'ST_EPIDEMIA', 'SK_LOCAL_OBITO', 'SK_TERMINAL_1', 'QTD_OBITOS'])

conn.commit()
conn.close()

In [22]:
df_fato_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345522 entries, 0 to 345521
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   TEMPO_MES        345522 non-null  object
 1   M.SK_ESTADO      345522 non-null  int64 
 2   SK_OCUPACAO      345522 non-null  int64 
 3   DS_SEXO          345522 non-null  object
 4   DS_FAIXA_ETARIA  345522 non-null  object
 5   ST_ASSISTENCIA   345522 non-null  object
 6   ST_INVESTIGACAO  345522 non-null  object
 7   ST_EPIDEMIA      345522 non-null  object
 8   SK_LOCAL_OBITO   345522 non-null  int64 
 9   SK_TERMINAL_1    345522 non-null  int64 
 10  QTD_OBITOS       345522 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 29.0+ MB


In [23]:
df_fato_covid.head()

,TEMPO_MES,M.SK_ESTADO,SK_OCUPACAO,DS_SEXO,DS_FAIXA_ETARIA,ST_ASSISTENCIA,ST_INVESTIGACAO,ST_EPIDEMIA,SK_LOCAL_OBITO,SK_TERMINAL_1,QTD_OBITOS
0,2020-01,13,-1,Feminino,Idoso,S,I,N,3,595,1
1,2020-03,1,2393,Masculino,Adulto,S,I,N,1,4378,1
2,2020-03,1,2795,Feminino,Idoso,S,I,N,1,595,1
3,2020-03,3,904,Masculino,Adulto,S,I,N,1,226,1
4,2020-03,3,1152,Masculino,Adulto,S,I,N,1,595,1


# Inserção da DIME_TEMPO_MES

In [24]:
# Conexão com o banco de destino (DM)
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Limpa a dimensão antes de recarregar
cursor.execute("DELETE FROM DIME_TEMPO_MES")

# Gera os registros distintos de TEMPO_MES
df_tempo = df_fato_perfil[['TEMPO_MES']].drop_duplicates().copy()

# Quebra o TEMPO_MES em ano e mês
df_tempo['NR_ANO'] = df_tempo['TEMPO_MES'].str.slice(0, 4).astype(int)
df_tempo['NR_MES'] = df_tempo['TEMPO_MES'].str.slice(5, 7).astype(int)

# SK_TEMPO_MES = YYYYMM como inteiro
df_tempo['SK_TEMPO_MES'] = (df_tempo['NR_ANO'].astype(str) +
                            df_tempo['NR_MES'].astype(str).str.zfill(2)).astype(int)

# Nome do mês
nomes_meses = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}
df_tempo['DS_MES'] = df_tempo['NR_MES'].map(nomes_meses)

# Trimestre e semestre
df_tempo['NR_TRIMESTRE'] = ((df_tempo['NR_MES'] - 1) // 3) + 1
df_tempo['NR_SEMESTRE'] = df_tempo['NR_MES'].apply(lambda x: 1 if x <= 6 else 2)

# Dias do mês
df_tempo['NR_QTD_DIAS_MES'] = pd.to_datetime(df_tempo['TEMPO_MES'] + "-01").dt.days_in_month

# Inserção na dimensão
for row in df_tempo.itertuples(index=False):
    cursor.execute("""
        INSERT INTO DIME_TEMPO_MES
        (SK_TEMPO_MES, DS_ANO_MES, NR_ANO, NR_MES, DS_MES,
         NR_TRIMESTRE, NR_SEMESTRE, NR_QTD_DIAS_MES, DT_CARGA)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row.SK_TEMPO_MES,
        row.TEMPO_MES,
        row.NR_ANO,
        row.NR_MES,
        row.DS_MES,
        row.NR_TRIMESTRE,
        row.NR_SEMESTRE,
        row.NR_QTD_DIAS_MES,
        data_carga
    ))

# Commit e fechar
conn.commit()
conn.close()



In [25]:
df_tempo.head(84)

,TEMPO_MES,NR_ANO,NR_MES,SK_TEMPO_MES,DS_MES,NR_TRIMESTRE,NR_SEMESTRE,NR_QTD_DIAS_MES
0,2018-01,2018,1,201801,Janeiro,1,1,31
61504,2018-02,2018,2,201802,Fevereiro,1,1,28
117702,2018-03,2018,3,201803,Março,1,1,31
180644,2018-04,2018,4,201804,Abril,2,1,30
243202,2018-05,2018,5,201805,Maio,2,1,31
...,...,...,...,...,...,...,...,...
5329762,2024-08,2024,8,202408,Agosto,3,2,31
5400836,2024-09,2024,9,202409,Setembro,3,2,30
5469698,2024-10,2024,10,202410,Outubro,4,2,31
5536827,2024-11,2024,11,202411,Novembro,4,2,30


# Inserindo na tabela DIME_JUNK_DETALHES

In [26]:
# Conexão com o banco de destino (DM)
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Limpa a dimensão antes de recarregar
cursor.execute("DELETE FROM DIME_JUNK_DETALHES")

# Gera os registros distintos da junk
df_detalhes = df_fato_impacto[['ST_ASSISTENCIA_MEDICA', 'ST_INVESTIGADO', 'ST_EPIDEMIA']].drop_duplicates().copy()

# Insere na tabela
for row in df_detalhes.itertuples(index=False):
    cursor.execute(
        """
        INSERT INTO DIME_JUNK_DETALHES
        (ST_ASSISTENCIA_MEDICA, ST_INVESTIGADO, ST_EPIDEMIA, DT_CARGA)
        VALUES ( ?, ?, ?, ?)
        """, (
            row.ST_ASSISTENCIA_MEDICA,
            row.ST_INVESTIGADO,
            row.ST_EPIDEMIA,
            data_carga
        )
    )

# Commit e fechar
conn.commit()
conn.close()


# Inserindo na tabela FATO_OBITO_COVID

In [27]:
# Data da carga
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Conectar no DM
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Limpa a dimensão antes de recarregar
cursor.execute("DELETE FROM FATO_OBITO_COVID")

# Iterar sobre o DataFrame e inserir na fato
for row in df_fato_covid.itertuples(index=False):

    # 1) Buscar SK_TEMPO_MES
    cursor.execute("SELECT SK_TEMPO_MES FROM DIME_TEMPO_MES WHERE DS_ANO_MES = ?", (row.TEMPO_MES,))
    sk_tempo = cursor.fetchone()
    if not sk_tempo:
        continue
    sk_tempo = sk_tempo[0]

    # 2) SK_ESTADO já vem do DW (precisa existir na DIME_ESTADO)
    sk_estado = row[1]

    # 3) SK_OCUPACAO
    sk_ocupacao = row.SK_OCUPACAO

    # 4) Buscar SK_DEMOGRAFICO (sexo + faixa etária)
    cursor.execute("""
        SELECT SK_DEMOGRAFICO
        FROM DIME_JUNK_DEMOGRAFICO
        WHERE DS_SEXO = ? AND DS_FAIXA_ETARIA = ?
    """, (row.DS_SEXO, row.DS_FAIXA_ETARIA))
    sk_demografico = cursor.fetchone()
    if not sk_demografico:
        continue
    sk_demografico = sk_demografico[0]

    # 5) Buscar SK_DETALHES (assistência, investigado, epidemia)
    cursor.execute("""
        SELECT SK_DETALHES
        FROM DIME_JUNK_DETALHES
        WHERE ST_ASSISTENCIA_MEDICA = ? AND ST_INVESTIGADO = ? AND ST_EPIDEMIA = ?
    """, (row.ST_ASSISTENCIA, row.ST_INVESTIGACAO, row.ST_EPIDEMIA))
    sk_detalhes = cursor.fetchone()
    if not sk_detalhes:
        continue
    sk_detalhes = sk_detalhes[0]

    # 6) SK_LOCAL_OBITO
    sk_local = row.SK_LOCAL_OBITO

    # 7) SK_TERMINAL_1
    sk_terminal = row.SK_TERMINAL_1

    # 8) Quantidade de óbitos
    qt_quantidade = row.QTD_OBITOS

    # Inserir na fato
    cursor.execute("""
        INSERT OR IGNORE INTO FATO_OBITO_COVID
        (SK_TEMPO_MES, SK_ESTADO, SK_OCUPACAO, SK_DEMOGRAFICO, SK_DETALHES, SK_LOCAL_OBITO, SK_TERMINAL_1, QT_QUANTIDADE, DT_CARGA)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (sk_tempo, sk_estado, sk_ocupacao, sk_demografico, sk_detalhes, sk_local, sk_terminal, qt_quantidade, data_carga))

# Commit e fechar
conn.commit()
conn.close()

print("Carga concluída com sucesso! 🚀")

Carga concluída com sucesso! 🚀


# Inserindo na FATO_OBITO_IMPACTO

In [28]:
# Data da carga
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Conectar ao DM_Banco
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Limpa a dimensão antes de recarregar
cursor.execute("DELETE FROM FATO_OBITO_IMPACTO")


# Iterar sobre o DataFrame e inserir na fato
for row in df_fato_impacto.itertuples(index=False):

    # 1) Buscar SK_TEMPO_MES
    cursor.execute("SELECT SK_TEMPO_MES FROM DIME_TEMPO_MES WHERE DS_ANO_MES = ?", (row.TEMPO_MES,))
    sk_tempo = cursor.fetchone()
    if not sk_tempo:
        continue
    sk_tempo = sk_tempo[0]

    # 2) Buscar SK_DETALHES (assistência, investigado, epidemia)
    cursor.execute("""
        SELECT SK_DETALHES
        FROM DIME_JUNK_DETALHES
        WHERE ST_ASSISTENCIA_MEDICA = ? AND ST_INVESTIGADO = ? AND ST_EPIDEMIA = ?
    """, (row.ST_ASSISTENCIA_MEDICA, row.ST_INVESTIGADO, row.ST_EPIDEMIA))
    sk_detalhes = cursor.fetchone()
    if not sk_detalhes:
        continue
    sk_detalhes = sk_detalhes[0]

    # 3) SK_CIRCUNSTANCIA (vem direto do DW, precisa existir na DIME_CIRCUNSTANCIA)
    sk_circunstancia = row.SK_CIRCUNSTANCIA

    # 4) SK_LOCAL_OBITO (vem direto do DW, precisa existir na DIME_LOCAL_OBITO)
    sk_local = row.SK_LOCAL_OBITO

    # 5) SK_TERMINAL (vem direto do DW, precisa existir na DIME_DOENCA)
    sk_terminal = row.SK_TERMINAL_1

    # 6) Quantidade de óbitos
    qt_quantidade = row.QTD_OBITOS

    # Inserir na fato
    cursor.execute("""
        INSERT INTO FATO_OBITO_IMPACTO
        (SK_TEMPO_MES, SK_DETALHES, SK_CIRCUNSTANCIA, SK_LOCAL_OBITO, SK_TERMINAL, QT_QUANTIDADE, DT_CARGA)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (sk_tempo, sk_detalhes, sk_circunstancia, sk_local, sk_terminal, qt_quantidade, data_carga))

# Commit e fechar
conn.commit()
conn.close()

print("Carga da FATO_OBITO_IMPACTO concluída com sucesso! 🚀")


Carga da FATO_OBITO_IMPACTO concluída com sucesso! 🚀


# Inserindo na tabela FATO_OBITO_PERFIL

In [29]:
# Data da carga
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Conectar ao DM_Banco
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Limpa a dimensão antes de recarregar
cursor.execute("DELETE FROM FATO_OBITO_PERFIL")

# Iterar sobre o DataFrame e inserir na fato
for row in df_fato_perfil.itertuples(index=False):

    # 1) Buscar SK_TEMPO_MES
    cursor.execute("SELECT SK_TEMPO_MES FROM DIME_TEMPO_MES WHERE DS_ANO_MES = ?", (row.TEMPO_MES,))
    sk_tempo = cursor.fetchone()
    if not sk_tempo:
        continue
    sk_tempo = sk_tempo[0]

    # 2) Buscar SK_DEMOGRAFICO (sexo + faixa etária)
    cursor.execute("""
        SELECT SK_DEMOGRAFICO
        FROM DIME_JUNK_DEMOGRAFICO
        WHERE DS_SEXO = ? AND DS_FAIXA_ETARIA = ?
    """, (row.DS_SEXO, row.DS_FAIXA_ETARIA))
    sk_demografico = cursor.fetchone()
    if not sk_demografico:
        continue
    sk_demografico = sk_demografico[0]

    # 3) SKs que já vêm prontos
    sk_terminal_1 = row.SK_TERMINAL_1
    sk_terminal_2 = row.SK_TERMINAL_2
    sk_basica_1   = row.SK_BASICA_1
    sk_basica_2   = row.SK_BASICA_2
    sk_estado     = row.SK_ESTADO

    # 4) Calcular ST_COVID
    st_covid = 1 if sk_basica_1 == 595 else 0

    # 5) Quantidade de óbitos
    qt_quantidade = row.QTD_OBITOS

    # Inserir na fato
    cursor.execute("""
        INSERT OR IGNORE INTO FATO_OBITO_PERFIL
        (SK_TEMPO_MES, SK_TERMINAL_1, SK_TERMINAL_2, SK_BASICA_1, SK_BASICA_2, SK_DEMOGRAFICO, SK_ESTADO, ST_COVID, QT_QUANTIDADE, DT_CARGA)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (sk_tempo, sk_terminal_1, sk_terminal_2, sk_basica_1, sk_basica_2,
          sk_demografico, sk_estado, st_covid, qt_quantidade, data_carga))

# Commit e fechar
conn.commit()
conn.close()

print("Carga da FATO_OBITO_PERFIL concluída com sucesso! 🚀")

Carga da FATO_OBITO_PERFIL concluída com sucesso! 🚀


# Salvando DIMES e FATOS como csv

## DIME_JUNK_DEMOGRAFICO

In [30]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_JUNK_DEMOGRAFICO
""")
demografico = cursor.fetchall()

df_demografico = pd.DataFrame(demografico)

conn.commit()
conn.close()

In [31]:
df_demografico.to_csv('DIME_JUNK_DEMOGRAFICO.csv', index=False)

## DIME_LOCAL_OBITO

In [32]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_LOCAL_OBITO
""")
local_obito = cursor.fetchall()

df_local_obito = pd.DataFrame(local_obito)

conn.commit()
conn.close()

In [33]:
df_local_obito.to_csv('DIME_LOCAL_OBITO.csv', index=False)

## DIME_OCUPACAO

In [34]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_OCUPACAO
""")
ocupacao = cursor.fetchall()

df_ocupacao = pd.DataFrame(ocupacao)

conn.commit()
conn.close()

In [35]:
df_ocupacao.to_csv('DIME_OCUPACAO.csv', index=False)

## DIME_DOENCA

In [36]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_DOENCA
""")
doenca = cursor.fetchall()

df_doenca = pd.DataFrame(doenca)

conn.commit()
conn.close()

In [37]:
df_doenca.to_csv('DIME_DOENCA.csv', index=False)

## DIME_CIRCUNSTANCIA

In [38]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_CIRCUNSTANCIA
""")
circunstancia = cursor.fetchall()

df_circunstancia = pd.DataFrame(circunstancia)

conn.commit()
conn.close()

In [39]:
df_circunstancia.to_csv('DIME_CIRCUNSTANCIA.csv', index=False)

## DIME_ESTADO

In [40]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_ESTADO
""")
estado = cursor.fetchall()

df_estado = pd.DataFrame(estado)

conn.commit()
conn.close()

In [41]:
df_estado.to_csv('DIME_ESTADO.csv', index=False)

## DIME_TEMPO_MES

In [42]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_TEMPO_MES
""")
tempo = cursor.fetchall()

df_tempo = pd.DataFrame(tempo)

conn.commit()
conn.close()

In [43]:
df_tempo.to_csv('DIME_TEMPO_MES.csv', index=False)

## DIME_JUNK_DETALHES

In [44]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM DIME_JUNK_DETALHES
""")
detalhes = cursor.fetchall()

df_detalhes = pd.DataFrame(detalhes)

conn.commit()
conn.close()

In [45]:
df_detalhes.to_csv('DIME_JUNK_DETALHES.csv', index=False)

## FATO_OBITO_COVID

In [46]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM FATO_OBITO_COVID
""")
obito_covid = cursor.fetchall()

df_obito_covid = pd.DataFrame(obito_covid)

conn.commit()
conn.close()

In [47]:
df_obito_covid.to_csv('FATO_OBITO_COVID.csv', index=False)

## FATO_OBITO_IMPACTO

In [48]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM FATO_OBITO_IMPACTO
""")
obito_impacto = cursor.fetchall()

df_obito_impacto = pd.DataFrame(obito_impacto)

conn.commit()
conn.close()

In [49]:
df_obito_impacto.to_csv('FATO_OBITO_IMPACTO.csv', index=False)

## FATO_OBITO_PERFIL

In [50]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM FATO_OBITO_PERFIL
""")
obito_perfil = cursor.fetchall()

df_obito_perfil = pd.DataFrame(obito_perfil)

conn.commit()
conn.close()

In [51]:
df_obito_perfil.to_csv('FATO_OBITO_PERFIL.csv', index=False)

##